In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

ModuleNotFoundError: No module named 'langchain_core.chains'

In [ ]:
import streamlit as st
import pandas as pd
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
#from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
#from langchain.agents import initialize_agent
from langchain.tools import tool
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [24]:
# --- 0. 初期設定 ---
st.set_page_config(page_title="ローカルRAGメモダッシュボード", layout="wide")
st.title("🧭 ローカルRAG + Agent メモダッシュボード")

# --- 1. 分類モデル ---
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# --- 2. LLMモデル（HF無料モデル使用） ---
@st.cache_resource
def load_llm():
    model_name = "google/flan-t5-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    summarizer = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
    return HuggingFacePipeline(pipeline=summarizer)

llm = load_llm()
llm.invoke("Hello, world!")
# --- 3. ベクトルストアの初期化 ---
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

if "vectorstore" not in st.session_state:
    st.session_state.vectorstore = FAISS.from_texts(["サンプルメモです"], embeddings)
if "memo_df" not in st.session_state:
    st.session_state.memo_df = pd.DataFrame(columns=["text", "category"])

# --- 4. メモ入力 ---
st.sidebar.header("📥 メモ入力")
text = st.sidebar.text_area("メモ or URLを入力")
categories = ["仕事", "勉強", "健康", "感情", "生活", "旅行", "その他"]

if st.sidebar.button("分類して追加"):
    if text.strip():
        res = classifier(text, candidate_labels=categories)
        best_cat = res["labels"][0]
        st.session_state.memo_df.loc[len(st.session_state.memo_df)] = [text, best_cat]
        st.session_state.vectorstore.add_texts([text])
        st.sidebar.success(f"カテゴリ: {best_cat} に登録しました ✅")
    else:
        st.sidebar.warning("メモを入力してください。")

# --- 5. メモ一覧 ---
st.subheader("📚 登録済みメモ")
st.dataframe(st.session_state.memo_df)

# --- 6. RAG検索＋Agent ---
retriever = st.session_state.vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

tools = [
    Tool(
        name="メモ検索ツール",
        func=qa_chain.run,
        description="メモデータから関連情報を検索して要約します"
    )
]

agent = initialize_agent(tools, llm, agent_type="zero-shot-react-description")

st.divider()
st.subheader("💬 Agentに質問してみよう")
query = st.text_input("例：「旅行カテゴリを要約して」など")
if st.button("実行"):
    answer = agent.run(query)
    st.write(answer)


2025-11-02 14:24:39.753 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-02 14:24:39.768 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-02 14:24:39.879 
  command:

    streamlit run /Users/woojoohyeon/Desktop/memo_categorizing/.venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-11-02 14:24:39.880 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-02 14:24:39.882 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
Device set to use mps:0
2025-11-02 14:24:59.900 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-02 14:24:59.904 Thread 'MainThread': mi

NameError: name 'RetrievalQA' is not defined